In [1]:
import os
import glob
import pandas as pd
import time
from tqdm import tqdm

from arcgis.gis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import SpatialDataFrame

develoersUser = 'yu_odanaka_ichilab'
develoersPass = 'odachu5515'
 
gis = GIS("https://www.arcgis.com",develoersUser,develoersPass)
user = gis.users.get(develoersUser)

In [2]:
#読むこむデータのURL末尾
url_dict={"2020":{
"東北":"9d643e62c970405f8c8d75e1552c6eb5",
"関東":"b895c1f1101f4502b92335f5793751b7",
"中部":"51665be6d3ac4a489a30452e1f0dd6e4",
"近畿":"5127884945c1402595af4428a7f1c173",
"中国":"f99976b3725140459dd682a485e273de",
"四国":"8baf011b1d854fd1846fd3965e61f73f",
"九州":"eacd9ee62c0148d6964bf27867ffc97f"
},
"2021":{
  "東北":"17865bfa8fb74b13ac2826a371945c80",
"近畿1":"96f67b5ce1af4db593f03357d71f11b4",
"近畿2":"7724b8b35537492b9e8a622079c856c3",
"中国":"25f6eea6ba954f47bc25b9eaf0ef0dca",
"四国":"47e464c737644dd79b6289b8bc6328d5",
"九州":"a42a134dbd294304bca0a45e6d7aeb7d",
"茨城":"64db1b4dec9b49d09b3769083c9d1e5c",
"栃木":"b84a5e104e174820b8bfe94faa9dd7bc",
"群馬":"3d5a75a87dac4ca7a652b1939b139c13",
"埼玉":"7ea8e736ad26487eb13f293b2dadc2fe",
"千葉":"621184ec7b8c47f9a916cf1d8cb393ad",
"東京":"1785f486c9c04b68bc846bdc15806a64",
"神奈川":"0a7892db2ff64435a004395d753f890b",
"新潟":"4fe955a1f0854e8082155653a2fa7a51",
"富山":"8d2dea74cd604db598cf5a754361712c",
"石川":"47950af5b8684fdc9b051912ee9d310c",
"福井":"113c97d4524e4c38a36fbbb307722d20",
"岐阜":"3835ce7c5ecd4cefa8240d696a4ac6bf",
"長野":"aa68f8b5adc2409faf9b281d0e2dc5a1",
"山梨":"f67d9453f45949308afb3bafbdfa75d4",
"愛知":"b4130db700b341e6ac446deacc7d0b19",
"静岡":"fc8aae1dd03a4c3aa360b01af702b279"  
}
}

In [4]:
bplayer_dict

{'東北': <FeatureLayer url:"https://services8.arcgis.com/3AhEzvrKsakNeCCe/arcgis/rest/services/東北_建物_人口メッシュ/FeatureServer/0">,
 '栃木': <FeatureLayer url:"https://services8.arcgis.com/3AhEzvrKsakNeCCe/arcgis/rest/services/関東_人口_建物/FeatureServer/1">,
 '茨城': <FeatureLayer url:"https://services8.arcgis.com/3AhEzvrKsakNeCCe/arcgis/rest/services/関東_人口_建物/FeatureServer/2">,
 '群馬': <FeatureLayer url:"https://services8.arcgis.com/3AhEzvrKsakNeCCe/arcgis/rest/services/関東_人口_建物/FeatureServer/3">,
 '埼玉': <FeatureLayer url:"https://services8.arcgis.com/3AhEzvrKsakNeCCe/arcgis/rest/services/関東_人口_建物/FeatureServer/4">,
 '東京': <FeatureLayer url:"https://services8.arcgis.com/3AhEzvrKsakNeCCe/arcgis/rest/services/関東_人口_建物/FeatureServer/5">,
 '千葉': <FeatureLayer url:"https://services8.arcgis.com/3AhEzvrKsakNeCCe/arcgis/rest/services/関東_人口_建物/FeatureServer/6">,
 '神奈川': <FeatureLayer url:"https://services8.arcgis.com/3AhEzvrKsakNeCCe/arcgis/rest/services/関東_人口_建物/FeatureServer/7">,
 '石川': <FeatureLayer url:"h

In [3]:
#建物人口の読み込み
bp_dict=url_dict["2020"]
item_dict={}
for name, key in bp_dict.items():
    item_dict[name]=gis.content.get(key)

    
#建物人口のlayer化
bplayer_dict={}
for i in item_dict.keys():
    #print(i)
    if i=="関東" or i == "中部" or i == "近畿":
        ls=item_dict[i].layers
        for l in ls:
            n=l.properties.name
            if "_Intersect" in n:
                n=n.replace("_Intersect","")
            elif  "_建物_人口" in n:
                n=n.replace("_建物_人口","")
            #print(n)
            bplayer_dict[n]=l
    else:
        bplayer_dict[i]=item_dict[i].layers[0]
bplayer_dict

#なぜか入ってしまう余計なものを削除
#del bplayer_dict["石川_建物_人口"], bplayer_dict["福井_建物_人口"],  bplayer_dict["長野_建物_人口"], bplayer_dict["静岡_建物_人口"], bplayer_dict["岐阜_建物_人口"], bplayer_dict["愛知_建物_人口"], bplayer_dict["富山_建物_人口"], bplayer_dict["山梨_建物_人口"], bplayer_dict["新潟_建物_人口"], bplayer_dict["近畿"]
#print(bplayer_dict)

##erro出てるけどその後の作業に問題なし

KeyError: '石川_建物_人口'

In [5]:
#21年500mメッシュ人口の読み込み
pl_dict=url_dict["2021"]
item_dict={}
for name, key in pl_dict.items():
    item_dict[name]=gis.content.get(key)

#21年500mメッシュ人口のlayer化
pllayer_dict={}
for name, item in item_dict.items():
    pllayer_dict[name]=item.layers[0]

In [7]:
#建物人口レイヤをsdfに変換し、辞書に格納
#最初の一つは動作確認済み
sdfbp_dict={}
geho=bplayer_dict.items()
for name,layer in geho:
    tmp=SpatialDataFrame.from_layer(layer)
    kiero=[]
    hoge=tmp.columns
    for i in hoge:
        if "2025" in i or "2030" in i or "2035" in i or "2040"  in i or "2045"  in i or "2050" in i or "Data" in i or "1st" in i or "FID" in i or "OBJECTID" in i:
            kiero.append(i)
        else:
            pass
    tmp=tmp.drop(kiero, axis=1)
    sdfbp_dict[name]=tmp

In [8]:
#21年人口をsdf化し、辞書に格納
sdf21_dict={}
huhu=pllayer_dict.items()
for name, layer in huhu:
    tmp=SpatialDataFrame.from_layer(layer)
    sdf21_dict[name]=tmp

In [14]:
#このあとで使うリストを定義（sdfに付け加えるものと最後に消すもの）
cn_list=["age_0",
     "age_3",
     "age_0_1",
     "age_0_2",
     "age_0_14",
     "age_6_12",
     "age_15_45_f",
     "age_15_50_f",
     "age_o13",
     "age_o65",
     "age_o75",
     "age_10_50",
     "age_40_64",
     "pop_total",
     'EST_INFANT',
       "EST_O_75",
      'EST_FOOD',
     'EST_LFOOD',
     'EST_AFOOD',
     'EST_WATER',
     'EST_BLNKT',
     'EST_PMILK',
     'EST_APMILK',
     'EST_BBTTL',
     'EST_PTILT',
     'EST_TILTPP',
     'EST_SANI',
     'EST_LIGHT',
     'EST_MKID',
     'EST_HKID',
     'EST_FALLGY',
     'EST_DIABTS',
     'EST_MENTAL',
     'EST_ME',
     'EST_MIMI',
     'EST_KETSU',
     'EST_NIN',
     'EST_KAIGO',
     'NUM'
      ]

kesu=["age_0",
 "age_3",
 "age_0_1",
 "age_0_2",
 "age_0_14",
 "age_6_12",
 "age_15_45_f",
 "age_15_50_f",
 "age_o13",
 "age_o65",
 "age_o75",
 "age_10_50",
 "age_40_64",
 "pop_total",
     "DATE",
     "NUM",
     "TYPE",
     "index_"]

In [12]:
#医療支援のデータの読み込みと整理
df_p = pd.read_csv("/Users/yuodanaka/Downloads/demand_simulator-master/resource/患者推計県番号順に外来_初診_再診_2.csv", index_col=0, encoding="shift_jis")
# 都道府県IDの作成
id_list = []
for i in range(1, 48):
    c = 0
    while c < (len(df_p) / 47):
        if i < 10:
            tmp = '0' + str(i)
            id_list.append(str(tmp))
        else:
            tmp = str(i)
            id_list.append(tmp)
        c += 1
df_p.insert(0, '都道府県ID', id_list)
# 都道府県IDを追加したデータフレーム保存後、再度読み込むと都道府県番号がint型になってしまう

# 平均診療間隔の呼び出し
df_i = pd.read_csv("/Users/yuodanaka/Downloads/demand_simulator-master/resource/平均診療間隔県番号順.csv", encoding="shift_jis", index_col=0)
# 都道府県IDの作成
id_list = []
for i in range(1, 48):
    c = 0
    while c < (len(df_i) / 47):
        if i < 10:
            tmp = '0' + str(i)
            id_list.append(str(tmp))
        else:
            tmp = str(i)
            id_list.append(tmp)
        c += 1
df_i.insert(0, '都道府県ID', id_list)

# 要介護者の呼び出し
df_c = pd.read_csv("/Users/yuodanaka/Downloads/demand_simulator-master/resource/都道府県要介護1号、2号順.csv", encoding="shift_jis", index_col=0)
# 都道府県IDの作成
id = id_list
df_c.insert(0, '都道府県ID', id * 2)
# print(df_c[df_c['都道府県ID']=='27'])

# 都道府県人口の呼び出し
df_pl = pd.read_csv("/Users/yuodanaka/Downloads/demand_simulator-master/resource/都道府県人口全部総人口、男、女別、都道府県番号順.csv", encoding="shift_jis", index_col=0)
# 都道府県IDの作成
id = id_list
df_pl.insert(0, '都道府県ID', id * 3)


#列名をアルファベット表記に変更
df_p.columns=["都道府県ID","diabe","mental","eye","ear","high","preg"]
df_i.columns=["都道府県ID","diabe","mental","eye","ear","high","preg"]

In [16]:
name_list= sdfbp_dict.keys()

for area_name in name_list:
    sdfbp=sdfbp_dict[area_name]
    sdf21=sdf21_dict[area_name]
    
    #500mメッシュに建物がいくつあるか
    mesh_list=sdfbp["MESH_ID"].to_list()
    mesh_num_dict={}
    mesh_unique=list(set(mesh_list))
    for i in mesh_unique:
        mesh_num_dict[i]=0

    for j in mesh_unique:
        for k in mesh_list:
            if j==k:
                mesh_num_dict[j]+=1

    #必要な列を持たせる
    for i in cn_list:
        sdfbp[i]=0

    #20年建物人口sdfを辞書型にして処理を軽くする
    dd=sdfbp.to_dict()
    #21年のsdfも辞書にする
    dd_21=sdf21.to_dict()

    #所属する500メッシュにいくつ建物が入っているかを持たせる
    mesh_dict=dd["MESH_ID"]

    for index, mesh_id in mesh_dict.items():
        for unique_mesh_id,num in mesh_num_dict.items():
            if mesh_id == unique_mesh_id:
                dd["NUM"][index]=num
            else:
                pass
    
    #元データに今後の計算で使う人口区分を追加
    len_bp=len(sdfbp)
    len_21=len(sdf21)

    for i in range(len_bp):
        #print(i)
        s=dd["NUM"][i]
        #print(s)
        if s == 0:
            pass
        else:
            for j in range(len_21):
                if dd["MESH_ID"][i]==dd_21["MESH_ID"][j]:

                    dd["age_0"][i]=round((dd_21["PT1_2021"][j]/5)/s, 20)
                    dd["age_3"][i]=round((dd_21["PT1_2021"][j]/5)/s,20)
                    dd["age_0_1"][i]=round((((dd_21["PT1_2021"][j]/5)*2)/s),20)
                    dd["age_0_2"][i]=round(((dd_21["PT1_2021"][j]/5)*3)/s, 20)
                    dd["age_0_14"][i]=round((dd_21["PTA_2021"][j])/s, 20)
                    dd["age_6_12"][i]=round(((dd_21["PT2_2021"][j]/5)*4+(dd_21["PT3_2021"][j]/5)*3)/s, 20)
                    dd["age_15_45_f"][i]=round((dd_21["PF4_2021"][j]+dd_21["PF5_2021"][j]+dd_21["PF6_2021"][j]+dd_21["PF7_2021"][j]+dd_21["PF8_2021"][j]+dd_21["PF9_2021"][j]+(dd_21["PF10_2021"][j]/5))/s, 20)
                    dd["age_15_50_f"][i]=round((dd_21["PF4_2021"][j]+dd_21["PF5_2021"][j]+dd_21["PF6_2021"][j]+dd_21["PF7_2021"][j]+dd_21["PF8_2021"][j]+dd_21["PF9_2021"][j]+dd_21["PF10_2021"][j])/s, 20)
                    dd["age_o13"][i]=round(((dd_21["PT3_2021"][j]/5)*2+dd_21["PTB_2021"][j]+dd_21["PTC_2021"][j])/s, 20)
                    dd["age_o75"][i]=round((dd_21["PTD_2021"][j])/s, 20)
                    dd["age_o65"][i]=round(dd_21["PTC_2021"][j]/s,20)
                    dd["age_40_64"][i]=round((dd_21["PT9_2021"][j]+dd_21["PT10_2021"][j]+dd_21["PT11_2021"][j]+dd_21["PT12_2021"][j]+dd_21["PT13_2021"][j])/s, 20)
                    dd["age_10_50"][i]=round((dd_21["PT3_2021"][j]+dd_21["PT4_2021"][j]+dd_21["PT5_2021"][j]+dd_21["PT6_2021"][j]+dd_21["PT7_2021"][j]+dd_21["PT8_2021"][j]+dd_21["PT9_2021"][j]+dd_21["PT10_2021"][j]+(dd_21["PT11_2021"][j]/5))/s, 20)
                    dd["pop_total"][i]=round((dd_21["PTN_2021"][j])/s, 20)

                else:
                    pass

    #物資需要の推計
    disaster_coefficient=0.65
    allergy_rate=0.02
    allergy_rate_baby=0.03
    al_rate=0.6

    for i in range(len_bp):
        count_older = dd['age_o75'][i]
        count_baby0 = dd['age_0'][i]
        count_baby1 = dd['age_0_1'][i]
        count_baby2 = dd['age_0_2'][i]
        count_lady = dd['age_10_50'][i]
        count_household = dd['pop_total'][i]
        count_all = dd['pop_total'][i]


        dd['EST_FOOD'][i] = round((count_all - count_baby1 - count_older) * 3 * 4 * (1-allergy_rate) * disaster_coefficient, 20)
        dd['EST_LFOOD'][i] = round((count_baby1 + count_older) * 3 * 4 * (1-allergy_rate) * disaster_coefficient, 20)
        dd['EST_AFOOD'] [i]= round((count_baby1 + count_older) * 3 * 4 * allergy_rate * disaster_coefficient, 20)
        dd['EST_WATER'][i] = round(count_all * 3 * 4 * disaster_coefficient, 20)
        dd['EST_BLNKT'][i] = round(count_all * disaster_coefficient, 20)
        dd['EST_PMILK'][i] = round(count_baby0 * al_rate * 110 * 4 * (1 - allergy_rate_baby) * disaster_coefficient, 20)
        dd['EST_APMILK'][i] = round(count_baby0 * al_rate * 110 * 4 * allergy_rate_baby * disaster_coefficient, 20)
        dd['EST_BBTTL'][i] = round(count_baby0 * al_rate * disaster_coefficient, 20)
        dd['EST_PTILT'][i] = round(count_all * 6 * 4 * disaster_coefficient, 20)
        dd['EST_TILTPP'][i] = round(count_all * 9 * 4 / 65 * disaster_coefficient, 20)
        dd['EST_SANI'][i] = round(count_lady * 5 / 31 * 25 * disaster_coefficient, 20)
        dd['EST_LIGHT'][i] = round(count_household * disaster_coefficient, 20)
        dd['EST_MKID'][i] = round(count_household * disaster_coefficient, 20)
        dd['EST_HKID'][i] = round(count_household * disaster_coefficient, 20)
        
        


    ##1建物ごとに需要を推計し辞書に追加
    for x in  range(len_bp):
        #上で呼び出したデータから使うものだけを取り出す
        y=str(dd["SHICODE"][x])


        N=y[:2]
        sick_list =sick_list=["diabe","mental","eye","ear","high","preg"]


        # 傷病別初診患者の辞書を作る
        np_dict = {}
        ddf = df_p[df_p['都道府県ID'] == N]
        df1 = ddf.iloc[0]

        for i in sick_list:
            tmp = float(df1[i])
            np_dict.setdefault(i, tmp * 1000)
            # np_dict.setdefault('初診:'+i, tmp*1000)

        # 傷病別再来患者の辞書を作る
        rp_dict = {}
        df2 = ddf.iloc[1]

        for i in sick_list:
            tmp = float(df2[i])
            #print(tmp)
            rp_dict.setdefault(i, tmp * 1000)

        # 傷病別入院患者の辞書を作る
        hp_dict = {}
        df3 = ddf.iloc[2]

        for i in sick_list:
            tmp = float(df3[i])

            hp_dict.setdefault(i, tmp * 1000)

        # 傷病別平均診療間隔
        interval_dict = {}
        dff = df_i[df_i['都道府県ID'] == N]

        for i in sick_list:
            tmp = float(dff[i])
            interval_dict.setdefault(i, tmp)

        # 要介護者データフレームの整理
        # 都道府県の要介護度別第1号第２号の数
        yokaigo_list = list(df_c.columns)
        yokaigo_list.pop(0)
        yokaigo_list.pop(5)
        yokaigo_list.pop(5)

        df_cc = df_c[df_c['都道府県ID'] == N]
        for i in range(len(df_cc)):
            for t in yokaigo_list:
                df_cc[t].values[i] = int(df_cc[t].values[i].replace(',', ''))


        ##計算
        ddf = df_pl[df_pl['都道府県ID'] == N]

        # 75歳以上人口の取得
        dd['EST_O_75'][x] = dd['age_o75'][x]

        # 乳児人口の取得
        dd['EST_INFANT'][x] = dd['age_0_1'][x]

        # 食物アレルギー保持者の取得
        h_01 = dd['age_0_1'][x]
        h_3 = dd['age_3'][x]
        h_612 = dd['age_6_12'][x]
        h_13_ = dd['age_o13'][x]

        h_allergie = h_01 * 0.1 + h_3 * 0.05 + h_612 * 0.03 + h_13_ * 0.01
        dd['EST_FALLGY'][x] = round(h_allergie, 20)

        #要配慮者数
        # 都道府県、症状ごとの要配慮者数
        t_youhairyosya = []
        for s in sick_list:
            tmp = round(hp_dict[s] + np_dict[s] + rp_dict[s] + (rp_dict[s] * interval_dict[s] * (6 / 7)), 20)
            t_youhairyosya.append(tmp)

        # 建物メッシュ人口
        h_p = dd['pop_total'][i]
        # 都道府県人口
        t_n = ddf['total'].values[0]#ここ要チェック
        # 妊婦は除外
        h_youhairyosya = []
        #print(t_youhairyosya)
        sick_name = ['DIABTS', 'MENTAL', 'ME', 'MIMI', 'KETSU']
        for s in range(len(sick_list) - 1):
            #print(s)
            if t_n == 0:
                tmp = 0
            else:
                tmp = round(t_youhairyosya[s] * (h_p / t_n), 20)
                #print(t_youhairyosya[s])
                #print(h_p / t_n)
                #print(t_youhairyosya[s] * (h_p / t_n))
            h_youhairyosya.append(tmp)

        for h, s in zip(h_youhairyosya, sick_name):
            dd['EST_' + s][x] = h

        # 都道府県ごと妊婦及び産婦
        # 妊婦及び産婦：要配慮者リスト番号5
        t_ninpu = round(t_youhairyosya[5], 20)
        #print(t_ninpu)
        # 建物メッシュ15-45歳女性人口
        h_p = dd['age_15_45_f'][x]
        #print(h_p)
        # 都道府県 15-45歳女性人口
        t_n = ddf['15〜45'].values[2]
        #print(t_n)

        #(t_ninpu*(h_p/t_n))
        if t_n == 0:
            h_ninpu = 0
        else:
            h_ninpu = round(t_ninpu * (h_p / t_n), 20)

        dd['EST_NIN'][x] = h_ninpu



        # 要介護者の計算
        # 1号保険者の要介護者数
        # 65歳以上建物メッシュ人口
        h_p_65 = dd['age_o65'][x]
        # 都道府県 65以上人口
        t_n_65 = ddf['65〜'].values[0]

        # 2号保険者の要介護者数
        # 40-64歳建物メッシュ人口
        h_p_4065 = dd['age_40_64'][x]
        # 都道府県 40-64人口
        t_n_4065 = ddf['40〜64'].values[0]

        # 避難所区域の要介護別の数
        #yokaigo_name = ['KAIGO1', 'KAIGO2', 'KAIGO3', 'KAIGO4', 'KAIGO5']
        kaigo_rank=[]
        for i in range(len(yokaigo_list)):
            if t_n_65 == 0:
                tmp1 = 0
            else:
                tmp1 = round(df_cc[yokaigo_list[i]].values[0] * (h_p_65 / t_n_65), 20)
            if t_n_4065 == 0:
                tmp2 = 0
            else:
                tmp2 = round(df_cc[yokaigo_list[i]].values[1] * (h_p_4065 / t_n_4065), 20)
            tmp = tmp1 + tmp2
            kaigo_rank.append(tmp)

        kaigo_total=sum(kaigo_rank)
        dd["EST_KAIGO"][x]=kaigo_total
        
    #再びsdf化
    sdf_new=pd.DataFrame(dd)
        
    for i in sdf_new.columns:
        if "2020" in i or "2015" in i:
            kesu.append(i)
    else:
        pass

    sdf_new=sdf_new.drop(kesu,axis=1)
    
    #layerにして保存
    layer_name=area_name+"_被災地需要_2021版"
    sdf_new.spatial.to_featurelayer(layer_name)
    
    print(area_name+": ok")


東北: ok
栃木: ok
茨城: ok
群馬: ok
埼玉: ok
東京: ok
千葉: ok
神奈川: ok
石川: ok
福井: ok
長野: ok
静岡: ok
岐阜: ok
愛知: ok
富山: ok
山梨: ok
新潟: ok
近畿2: ok
近畿1: ok


KeyboardInterrupt: 